# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

1.Assign each point to a cluster N at random.
2.Calculate the mean position of each cluster using the previous assignments.
3.Loop through the points - assign each point to the cluster to whose center it is closest.
4.Repeat this process until the centers stop moving around.

#### b) Create the builder function for step 1.

In [12]:
label_randomly <- function(n_points, n_clusters){
  sample(1:n_clusters, n_points, replace = TRUE)
}

#### c) Create the builder function for step 2.

In [ ]:
get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(-label__, mean), .groups = "drop") %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [11]:
assign_cluster <- function(data, means){
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()
  for (point_index in dii){
    smallest_dist <- Inf
    smallest_label <- NA
    point <- as.numeric(data[point_index, ])
    for (clus_index in cii){
      clus_vec <- as.numeric(means[clus_index, colnames(means) != "label__"])
      diff <- point - clus_vec
      dist <- sum(diff * diff)

      if (dist < smallest_dist){
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }

    labels <- c(labels, smallest_label)
  }

  labels
}


#### e) Create the builder function for step 4.

In [10]:
kmeans_done <- function(old_means, new_means, eps = 1e-6){
  om <- as.matrix(old_means[, colnames(old_means) != "label__"])
  nm <- as.matrix(new_means[, colnames(new_means) != "label__"])
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if (m < eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [22]:
mykmeans <- function(data, n_clusters, eps = 1e-6){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE

  while (!done){
    labels <- assign_cluster(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if (kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
    }
  }

  list(labels = labels, means = new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [23]:
df <- read.csv("voltages_df.csv")

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [26]:
library(dplyr)
df <- read.csv("voltages_df.csv")

label_randomly <- function(n_points, n_clusters){
  sample(1:n_clusters, n_points, replace = TRUE)
}

get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

assign_cluster <- function(data, means){
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()

  for (point_index in dii){
    smallest_dist <- Inf
    smallest_label <- NA

    point <- as.numeric(data[point_index, ])

    for (clus_index in cii){
      clus_vec <- as.numeric(means[clus_index, colnames(means) != "label__"])
      diff <- point - clus_vec
      dist <- sum(diff * diff)
      if (dist < smallest_dist){
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }
    labels <- c(labels, smallest_label)
  }
  labels
}

kmeans_done <- function(old_means, new_means, eps = 1e-6){
  om <- as.matrix(old_means[, colnames(old_means) != "label__"])
  nm <- as.matrix(new_means[, colnames(new_means) != "label__"])
  m <- mean(sqrt(rowSums((om - nm)^2)))
  m < eps
}

mykmeans <- function(data, n_clusters, eps = 1e-6){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE

  while (!done){
    labels <- assign_cluster(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if (kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
    }
  }

  list(labels = labels, means = new_means)
}

X <- df[, sapply(df, is.numeric), drop = FALSE]

set.seed(1)
results <- mykmeans(X, n_clusters = 3)
cat("results$labels:\n")
print(results$labels)
cat("\nresults$means:\n")
print(results$means)

results$labels:
  [1] 1 2 2 2 2 2 3 3 2 1 1 1 3 1 2 1 1 1 1 1 2 2 2 2 2 1 1 1 3 3 3 3 2 3 3 2 2
 [38] 1 3 2 1 1 1 3 2 3 3 2 3 3 2 3 1 1 3 1 2 1 3 2 1 1 2 1 3 3 2 3 1 3 1 2 3 3
 [75] 2 2 1 3 3 3 3 3 2 3 3 2 2 2 1 3 3 3 2 3 2 2 3 2 2 3 3 3 2 3 1 1 3 1 1 2 1
[112] 1 2 3 1 3 1 1 2 2 2 2 1 1 3 2 2 3 1 3 2 1 2 3 1 2 2 2 1 3 1 2 2 3 1 1 2 1
[149] 1 3 1 3 2 2 3 1 3 3 2 1 3 3 1 2 1 3 2 1 2 1 1 3 2 1 3 3 2 3 1 2 3 2 3 1 3
[186] 2 2 1 2 1 1 2 1 1 3 3 1 2 1 1 3 1 3 3 2 1 2 3 2 1 2 3 1 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 3 3 3 1 2 1 3 2 2 3 3 1 1 3 3 1 3 3 2 3 1 3 3 2 3 2 1 3 2 3 3 1 1
[260] 1 2 2 3 1 2 2 2 3 3 3 1 2 2 2 3 1 3 3 2 2 3 1 3 1 1 2 3 2 2 3 3 1 3 3 1 2
[297] 1 3 3 2 2 3 1 1 1 3 1 2 2 2 1 1 1 1 1 1 2 1 2 1 1 3 2 3 1 1 1 1 2 1 2 1 3
[334] 1 2 2 3 3 1 2 3 2 3 3 1 3 2 3 1 1 3 2 2 2 1 3 2 1 3 3 2 2 2 2 1 3 1 2 1 1
[371] 1 1 1 2 2 3 3 1 1 1 1 3 3 1 1 3 3 2 2 3 1 2 2 2 2 3 2 2 3 1 1 1 1 2 2 1 1
[408] 3 3 3 2 3 3 1 2 2 2 3 2 2 2 1 1 2 1 3 2 2 2 2 3 3 2 3 1 2 1 1 1 3 3 3 3 1
[445] 2 1 1 1 3 2 1 1 1 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [27]:
df <- read.csv("voltages_df.csv")
X <- as.matrix(df)

set.seed(1)
km <- kmeans(X, centers = 3)
results <- list(
  labels  = km$cluster,
  cluster = km$cluster
)
cat("results$labels:\n")
print(results$labels)
cat("\nresults$cluster:\n")
print(results$cluster)

results$labels:
  [1] 1 2 2 2 2 2 3 3 2 1 1 1 3 1 2 1 1 1 1 1 2 2 2 2 2 1 1 1 3 3 3 3 2 3 3 2 2
 [38] 1 3 2 1 1 1 3 2 3 3 2 3 3 2 3 1 1 3 1 2 1 3 2 1 1 2 1 3 3 2 3 1 3 1 2 3 3
 [75] 2 2 1 3 3 3 3 3 2 3 3 2 2 2 1 3 3 3 2 3 2 2 3 2 2 3 3 3 2 3 1 1 3 1 1 2 1
[112] 1 2 3 1 3 1 1 2 2 2 2 1 1 3 2 2 3 1 3 2 1 2 3 1 2 2 2 1 3 1 2 2 3 1 1 2 1
[149] 1 3 1 3 2 2 3 1 3 3 2 1 3 3 1 2 1 3 2 1 2 1 1 3 2 1 3 3 2 3 1 2 3 2 3 1 3
[186] 2 2 1 2 1 1 2 1 1 3 3 1 2 1 1 3 1 3 3 2 1 2 3 2 1 2 3 1 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 3 3 3 1 2 1 3 2 2 3 3 1 1 3 3 1 3 3 2 3 1 3 3 2 3 2 1 3 2 3 3 1 1
[260] 1 2 2 3 1 2 2 2 3 3 3 1 2 2 2 3 1 3 3 2 2 3 1 3 1 1 2 3 2 2 3 3 1 3 3 1 2
[297] 1 3 3 2 2 3 1 1 1 3 1 2 2 2 1 1 1 1 1 1 2 1 2 1 1 3 2 3 1 1 1 1 2 1 2 1 3
[334] 1 2 2 3 3 1 2 3 2 3 3 1 3 2 3 1 1 3 2 2 2 1 3 2 1 3 3 2 2 2 2 1 3 1 2 1 1
[371] 1 1 1 2 2 3 3 1 1 1 1 3 3 1 1 3 3 2 2 3 1 2 2 2 2 3 2 2 3 1 1 1 1 2 2 1 1
[408] 3 3 3 2 3 3 1 2 2 2 3 2 2 2 1 1 2 1 3 2 2 2 2 3 3 2 3 1 2 1 1 1 3 3 3 3 1
[445] 2 1 1 1 3 2 1 1 1 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

Looking at both labels and clusters tables, they apear to be very similar. When looking at the means they also look simiar to the kmeans. Overall, data looks consistant.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

Using a for loop means iterating every point in order to assign a cluster for kmeans. What the for loop does is calculte the distance between each center and recored the closesed centers index and insert it into the labels.Once the loop is finished the labels variable has a cluster for every given point. After the labels are obtained, the means are calculated.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

You obtain all the centers and for all of the points you compute the distances to all the centers. Then, for each point take the minimum value of the centers which becomes the new cluster label. Using this methods increases latency of the program.  

#### c) State which (for loops or vectorizing) is more efficient and why.

Vectoizing is more efficent because for the for loop, the for loop has to go through all the points in the dataset and while this may be okay for small data sets, most data sets have lots of points which would make the code run slower. Whereas if the dataset is vectorized it is put into a matrix allowing for other operations done for k means to be done quicker.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

K means works best when the datapoints are close to each other and when clusters are roundish, esentially when clusters are around the same size. If k means isn't presenting that way then you might get misleading results and wrong interpretation of the data. An assumption that kmeans uses is that assumes you have vectorial data which is  distributed into uniformly shaped Gaussian of similar size and shape. It also assumes that an element is either in a cluster or not.

## Question 5
#### What assumption do Guassian mixture models make?

They make the assumption that the dataset is done by mixing k shaped clouds of data and that each cloud has its own mean and shape. So while kmeans says to pick the nearest center, the guassian mixture model says to give them a fuzzy k-means which gives the model a more softer assignment rather than one that has to resember a circle.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

The assumption that spectral clustering makes is that two points are more likely to be in the same cluster if they are close. This helps because it helps organize more messy data and makes it easier to interpret/organize the data. Additionally, this requires a similarity matrix instead of a guassian or vector space one. The data can be turned into a graph and eigenvectors to embed points into a low-dimensional connectivity space where clusters separate cleanly.   

## Question 7
#### Define the gap statistic method. What do we use it for?

Using data we can pick the number of clusters by comparing how tight the clusters are to what the expected random data is from the same state. This is used to chose K in clustering so see how many clusters we will have.